# Step 2. Recommendation Engines

1. We're using Turicreate library to create the recommenders.
2. We then input the recommendation engine with ratings score and sia
(compound)scores and compare the results.

### Note: Run the follwoing in python 3.7

In [147]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from matplotlib import cm

In [148]:
import turicreate as tc
from turicreate import SFrame

In [172]:
df_vaders = pd.read_csv ('/Users/mustansirmurtazakaj/Desktop/pythonjupyter/Amazonreviews/vaders.csv')
df_vaders.to_excel('/Users/mustansirmurtazakaj/Desktop/pythonjupyter/Amazonreviews/vaders_raw.xlsx')
df_vaders.describe()

#df_vaders

,Unnamed: 0,Id,neg,neu,pos,compound,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,5.000000e+02
mean,249.500000,250.500000,0.041372,0.765020,0.193614,0.638864,0.952000,1.27600,4.316000,1.294820e+09
std,144.481833,144.481833,0.053273,0.104938,0.109269,0.460826,2.045988,2.48922,1.202929,5.072437e+07
min,0.000000,1.000000,0.000000,0.355000,0.000000,-0.972600,0.000000,0.00000,1.000000,1.107821e+09
25%,124.750000,125.750000,0.000000,0.701000,0.119750,0.570900,0.000000,0.00000,4.000000,1.267790e+09
50%,249.500000,250.500000,0.027000,0.771500,0.180000,0.842600,0.000000,0.00000,5.000000,1.312978e+09
75%,374.250000,375.250000,0.064250,0.839000,0.260250,0.938100,1.000000,2.00000,5.000000,1.334621e+09
max,499.000000,500.000000,0.344000,1.000000,0.645000,0.995700,19.000000,19.00000,5.000000,1.351210e+09


In [176]:
sf_vaders=tc.SFrame(df_vaders)
#split into test and rain datasets
sftrain_vaders, sftest_vaders = sf_vaders.random_split(0.8, seed=42)

In [177]:
#modelling
## User similarity
collab_model_rating=tc.recommender.item_similarity_recommender.create(sftrain_vaders, user_id='Id', item_id='ProductId', target='Score', user_data=None, item_data=None, nearest_items=None, similarity_type='jaccard', threshold=0.001, only_top_k=64, verbose=True, target_memory_usage=8589934592)
collab_model_sia=tc.recommender.item_similarity_recommender.create(sftrain_vaders, user_id='Id', item_id='ProductId', target='compound', user_data=None, item_data=None, nearest_items=None, similarity_type='jaccard', threshold=0.001, only_top_k=64, verbose=True, target_memory_usage=8589934592)

Warning: Ignoring columns Unnamed: 0, neg, neu, pos, compound, UserId, ProfileName, HelpfulnessNumerator, HelpfulnessDenominator, Time, Summary, Text;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 396 observations with 396 users and 121 items.

Data prepared in: 0.009683s

...

| 4.559ms                             | 100              | 121             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.005652s

In [162]:
collab_recom_rating = collab_model_rating.recommend(users=None, k=5, exclude=None, items=None, new_observation_data=None, new_user_data=None, new_item_data=None, exclude_known=True, diversity=0, random_seed=None, verbose=True)
collab_recom_compound = collab_model_sia.recommend(users=None, k=5, exclude=None, items=None, new_observation_data=None, new_user_data=None, new_item_data=None, exclude_known=True, diversity=0, random_seed=None, verbose=True)

In [179]:
# conmverting back to df
vaders_recomm_rating = pd.DataFrame(collab_recom_rating)
vaders_recomm_rating = vaders_recomm_rating.reset_index().rename(columns={'rank': 'rating_rank'})
vaders_recomm_rating.drop(columns=['score', 'index'], inplace=True)

vaders_recomm_compound = pd.DataFrame(collab_recom_compound)
vaders_recomm_compound = vaders_recomm_compound.reset_index().rename(columns={'rank': 'compound_rank'})
vaders_recomm_compound.drop(columns=['score', 'index'], inplace=True)


In [185]:
collab_model_rating.evaluate(sftest_vaders, metric='auto')


Precision and recall summary statistics by cutoff
+--------+-----------------------+----------------------+
| cutoff |     mean_precision    |     mean_recall      |
+--------+-----------------------+----------------------+
|   1    |          0.0          |         0.0          |
|   2    |          0.0          |         0.0          |
|   3    |          0.0          |         0.0          |
|   4    | 0.0024038461538461544 | 0.009615384615384618 |
|   5    | 0.0019230769230769232 | 0.009615384615384618 |
|   6    | 0.0016025641025641025 | 0.009615384615384618 |
|   7    | 0.0013736263736263735 | 0.009615384615384618 |
|   8    | 0.0024038461538461535 | 0.01923076923076923  |
|   9    |  0.002136752136752137 | 0.01923076923076923  |
|   10   | 0.0019230769230769236 | 0.01923076923076923  |
+--------+-----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 3.913309565849558

Per User RMSE (best)
+-----+--------------------+-------+
|  Id |        rmse   

{'precision_recall_by_user': Columns:
 	Id	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 1872
 
 Data:
 +-----+--------+-----------+--------+-------+
 |  Id | cutoff | precision | recall | count |
 +-----+--------+-----------+--------+-------+
 | 145 |   1    |    0.0    |  0.0   |   1   |
 | 145 |   2    |    0.0    |  0.0   |   1   |
 | 145 |   3    |    0.0    |  0.0   |   1   |
 | 145 |   4    |    0.0    |  0.0   |   1   |
 | 145 |   5    |    0.0    |  0.0   |   1   |
 | 145 |   6    |    0.0    |  0.0   |   1   |
 | 145 |   7    |    0.0    |  0.0   |   1   |
 | 145 |   8    |    0.0    |  0.0   |   1   |
 | 145 |   9    |    0.0    |  0.0   |   1   |
 | 145 |   10   |    0.0    |  0.0   |   1   |
 +-----+--------+-----------+--------+-------+
 [1872 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.,
 'precision_recall_overall': Columns:
 	cutoff	int
 	preci

In [187]:
collab_model_sia.evaluate(sftest_vaders, metric='auto')


Precision and recall summary statistics by cutoff
+--------+-----------------------+----------------------+
| cutoff |     mean_precision    |     mean_recall      |
+--------+-----------------------+----------------------+
|   1    |          0.0          |         0.0          |
|   2    |          0.0          |         0.0          |
|   3    |          0.0          |         0.0          |
|   4    |          0.0          |         0.0          |
|   5    | 0.0019230769230769234 | 0.009615384615384616 |
|   6    |  0.003205128205128205 | 0.01923076923076923  |
|   7    |  0.002747252747252747 | 0.019230769230769232 |
|   8    | 0.0024038461538461535 | 0.01923076923076923  |
|   9    | 0.0021367521367521365 | 0.019230769230769232 |
|   10   |  0.002884615384615385 | 0.028846153846153848 |
+--------+-----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.5722663537457745

Per User RMSE (best)
+----+------+-------+
| Id | rmse | count |
+----+------+-

{'precision_recall_by_user': Columns:
 	Id	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 1872
 
 Data:
 +-----+--------+-----------+--------+-------+
 |  Id | cutoff | precision | recall | count |
 +-----+--------+-----------+--------+-------+
 | 376 |   1    |    0.0    |  0.0   |   1   |
 | 376 |   2    |    0.0    |  0.0   |   1   |
 | 376 |   3    |    0.0    |  0.0   |   1   |
 | 376 |   4    |    0.0    |  0.0   |   1   |
 | 376 |   5    |    0.0    |  0.0   |   1   |
 | 376 |   6    |    0.0    |  0.0   |   1   |
 | 376 |   7    |    0.0    |  0.0   |   1   |
 | 376 |   8    |    0.0    |  0.0   |   1   |
 | 376 |   9    |    0.0    |  0.0   |   1   |
 | 376 |   10   |    0.0    |  0.0   |   1   |
 +-----+--------+-----------+--------+-------+
 [1872 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.,
 'precision_recall_overall': Columns:
 	cutoff	int
 	preci

The two root mean square error (RMSE) values indicate that the collaborative filtering model constructed using sentiment intensity analyzer (SIA) scores exhibits an 85% reduction in error compared to the model constructed using product ratings.